<center><h1> Truss Investment </h1></center>
<center><h1> Trades Data Analysis Report</h1></center>
  
### April 2021 
---

In [7]:
##### LIBRARIES AND FUNCTIONS#####

# Load libraries
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Layout, interact, interactive
from IPython.display import clear_output, display, HTML
import datetime as dt
from datetime import datetime, date
import plotly.express as px
import plotly.graph_objects as go

import ctypes
from ctypes import *
#import pymsgbox

# Import functions
from functions import *
      
# Display options 
pd.set_option('display.max_rows', 500) # all rows
pd.options.mode.chained_assignment = None # spress warning
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [8]:
print('\033[93m' + '\033[1m' + "Please Select a Trade")

Please Select a Trade


In [9]:
##### READ DATASETS #####

# Get trades dataset list
dfs_list = Get_Trades()

# Read receipt df
BR_receipt = pd.read_csv('BR Receipt.csv')
US_futures_receipt = pd.read_csv('US Futures Receipt.csv')
all_trades = pd.read_csv('All Trades Receipt.csv')

In [10]:
##### DATE PICKER #####

# Get start and end date
today = date.today()
begin_date = dt.datetime(2020, 7, 1).date()

# Create begin date picker
begin_period = widgets.DatePicker(description='Start:',
                                 value = begin_date)

# Create end date picker
end_period = widgets.DatePicker(description='End:',
                               value = today)

In [11]:
# Function to get data for BR_trades
def Trade_Select(trade):
    
    for i in range(0, len(dfs_list)):
        
        if (dfs_list[i]['Ticker'][0] == trade):
            
            return(dfs_list[i])

In [12]:
##### BRAZIL TRADES DROPDOWN MENU #####

# Create dropdown menu to selct trade
BR_trade_dropdown = interactive(Trade_Select,
                                trade = widgets.Combobox(
                                    options= ['DOLG21',
                                              'DOLQ20',
                                              'DOLU20',
                                              'INDG21',
                                              'INDJ21',
                                              'INDM21',
                                              'ISPH21',
                                              'ISPM21',
                                              'ISPU20',
                                              'ISPU21',
                                              'ISPZ20',
                                              'WDOF21',
                                              'WDOG21',
                                              'WDOJ21',
                                              'WDOK21',
                                              'WDOM21',
                                              'WDOQ20',
                                              'WDOU20',
                                              'WING21',
                                              'WINJ21',
                                              'WINM21',
                                              'WSPH21',
                                              'WSPM21',
                                              'WSPU20',
                                              'WSPZ20'],
                                    description="Trade:")
                               )

In [13]:
##### BRAZIL TRADES OPERATIONAL CHECKBOX #####

# Create vector with operations
data = ["Buy Quant", "Buy Price", "Sell Quant", "Sell Price", "Px Close"]

# Create checkbox operations
BR_checkboxes_operation = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox operations horizontal box
BR_hbox_operation = widgets.HBox(children=BR_checkboxes_operation, layout=Layout(width='600px'))

In [14]:
##### BRAZIL TRADES FINANCIAL CHECKBOX #####

# Create vector with financial
data = ["Hold Pos", "MtoM P&L", "Gross P&L", "Comm", "Exc Fee", "Net P&L"]

# Create checkbox financial
BR_checkboxes_financial = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox financial horizontal box
BR_hbox_financial = widgets.HBox(children=BR_checkboxes_financial, layout=Layout(width='700px'))

In [15]:
##### BRAZIL TRADES BUTTONS #####

# Create button and output
BR_trades_button = Button_Sumbmit()

# Create button output
BR_trades_output = widgets.Output()

# Create button and output
BR_trades_clean = Button_Clean()

# Create button output
BR_trades_output_clean = widgets.Output()

# Unite buttons
BR_trades_ui = widgets.HBox([BR_trades_button, BR_trades_clean])

In [16]:
##### BRAZIL TRADES BUTTONS FUNCTIONS #####

# Create button clicked function
def BR_Trades_Clicked(b):  
    with BR_trades_output:
        
        # Clean output
        clear_output()
        
        if (BR_trade_dropdown.result is None):
            
            # Display warning message box
            ctypes.windll.user32.MessageBoxW(0, "Please select a trade!", "Error", 0x10)
            #pymsgbox.alert('This is an alert!', 'Title')
       
        else:
        
            # Create selected_data list
            selected_data = []
        
            # Fixede variables
            fixed = ['Date', 'Ticker']
        
            # Add fixed variables
            selected_data = selected_data + fixed
                    
            # Loop operation filter
            for i in range(0, len(BR_checkboxes_operation)):
                    if BR_checkboxes_operation[i].value == True:
                        selected_data = selected_data + [BR_checkboxes_operation[i].description]
        
            # Loop financial filter
            for j in range(0, len(BR_checkboxes_financial)):
                    if BR_checkboxes_financial[j].value == True:
                        selected_data = selected_data + [BR_checkboxes_financial[j].description]
                    
            # Create trade dataframe
            df = BR_trade_dropdown.result
            
            # Round values
            df = df.round({'Px Close':2, 'Sell Price':2, 'Buy Price': 2, 'Exc Fee':2, 'MtoM P&L':2, 'Gross P&L': 2, 'Net P&L': 2, 'Comm': 2})
        
            # Change columns type
            df = df.astype({'Sell Quant': int, 'Buy Quant': int, 'Hold Pos': int})
        
            # Filter data
            df = df[selected_data] 
               
            # Transform into date type
            df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
        
            # Apply filter
            df = df[(df['Date'] >= pd.Timestamp(begin_period.value)) &  (df['Date'] <= pd.Timestamp(end_period.value))]   
        
            # Sort by date
            df = df.sort_values(by = 'Date')
        
            # Refine date
            df['Date'] = df['Date'].dt.strftime('%d/%m/%Y')

            # Reset index
            df = df.reset_index(drop = True)
        
            #Create red columns vector
            red_col = ['MtoM P&L' , 'Gross P&L', 'Net P&L', 'Hold Pos']
       
            # Create auxiliar variable
            aux = []
        
            # Loop to define red columns
            for i in range(0,len(red_col)):
                if(red_col[i] in df.columns):
                    aux.append(red_col[i])
                    
            if aux == []:
                
                # Print result (without red columns)
                display(df.style.set_precision(2))
                
            else:
            
                # Print result (with red columns)
                display(df.style.applymap(color_negative_red, subset=pd.IndexSlice[:, aux]).set_precision(2))
            
# Create clean button function
def BR_Trades_Clean_Clicked(b):  
    with BR_trades_output_clean:
        
        # Clean output
        BR_trades_output.clear_output()

In [17]:
##### BRAZIL TRADES TABLE ON BUTTON CLIKED #####

# On button click
BR_trades_button.on_click(BR_Trades_Clicked)
BR_trades_clean.on_click(BR_Trades_Clean_Clicked)

In [18]:
###### BRAZIL TRADES VISUALIZATION BUTTONS #####

# Create button and output
BR_trades_viz_button = Button_Sumbmit()

# Create button output
BR_trades_viz_output = widgets.Output()

# Create button and output
BR_trades_viz_clean = Button_Clean()

# Create button output
BR_trades_viz_output_clean = widgets.Output()

# Unite buttons
BR_trades_viz_ui = widgets.HBox([BR_trades_viz_button, BR_trades_viz_clean])

In [19]:
##### BRAZIL TRADES VISUALIZATION BUTTONS FUNCTIONS #####

# Create button clicked function
def BR_Trades_Viz_Clicked(b):  
    with BR_trades_viz_output:
        
        # Clean output
        clear_output()
 
        # Create trade dataframe
        df = BR_trade_dropdown.result
        
        # Transform into date type
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
        
        # Apply filter
        df = df[(df['Date'] >= pd.Timestamp(begin_period.value)) &  (df['Date'] <= pd.Timestamp(end_period.value))]   
        
        # Transform df format
        df_plot = pd.melt(df[['Buy Quant', 'Sell Quant']])

        # Concat Date
        df_plot['Date'] = pd.concat([df['Date'], df['Date']]).reset_index(drop = True)
        
        # Concat Px Close Diff
        df_plot['Px Close Diff'] = pd.concat([abs(df['Px Close'].diff()), abs(df['Px Close'].diff())]).reset_index(drop = True)
        
        # Plot graph
        fig = px.bar(df_plot, x= "Date", y= "value", color= "variable", title= 'The plot shows trade ') 

        # Update fig layout
        fig.update_layout(barmode= 'group')
        fig.update_yaxes(title='')
        
        # Add close price difference
        fig.add_trace(go.Scatter(mode= "lines", x= df_plot["Date"], y= df_plot["Px Close Diff"], name= "Close Variance"))
        
        # Plot graph
        fig.show()
        
# Create clean button function
def BR_Trades_Viz_Clean_Clicked(b):  
    with BR_trades_viz_output_clean:
        
        # Clean output
        BR_trades_viz_output.clear_output()

In [20]:
##### BRAZIL TRADES VISUALIZATION ON BUTTON CLIKED #####

# On button click
BR_trades_viz_button.on_click(BR_Trades_Viz_Clicked)
BR_trades_viz_clean.on_click(BR_Trades_Viz_Clean_Clicked)

In [21]:
##### BRAZIL RECEIPT CHECKBOX #####

# Create vector with checkbox data
data = ['Receipt Value', 'FX Description', 'FX Value', 'FX Rate', 'Comm', 'Exc Fee', 'Reg Fee']

# Create checkbox receipt
BR_receipt_checkboxes = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox financial horizontal box
BR_receipt_hbox = widgets.HBox(children=BR_receipt_checkboxes, layout=Layout(width='900px'))

In [22]:
##### BRAZIL RECEIPT CUMSUM CHECKBOX #####

# Create vector with cumsum
data = ['Receipt Value Cum Sum', 'FX Value Cum Sum', 'Total Cum Sum', 'Settlement Cum Sum']

# Create checkbox cumsum
BR_receipt_checkboxes_cumsum = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox cumsum horizontal box
BR_receipt_hbox_cumsum = widgets.HBox(children=BR_receipt_checkboxes_cumsum, layout=Layout(width='700px'))

In [23]:
##### BRAZIL RECEIPT BUTTONS #####

# Create button and output
BR_receipt_button = Button_Sumbmit()

# Create button output
BR_receipt_output = widgets.Output()

# Create button and output
BR_receipt_clean = Button_Clean()

# Create button output
BR_receipt_output_clean = widgets.Output()

# Unite buttons
BR_receipt_ui = widgets.HBox([BR_receipt_button, BR_receipt_clean])

In [24]:
##### BRAZIL RECEIPT BUTTONS FUNCTIONS #####

# Create button clicked function
def BR_Receipt_Clicked(b):
    with BR_receipt_output:
        
        # Clean output
        clear_output()
        
        # Create receipt dataframe
        df = BR_receipt
        
        # Create selected_data list
        selected_data = []
        
        # Add fixed variables
        selected_data = selected_data + ['Date']
                    
        # Loop operation filter
        for i in range(0, len(BR_receipt_checkboxes)):
                if BR_receipt_checkboxes[i].value == True:
                    selected_data = selected_data + [BR_receipt_checkboxes[i].description]
                    
        # Loop operation filter
        for j in range(0, len(BR_receipt_checkboxes_cumsum)):
                if BR_receipt_checkboxes_cumsum[j].value == True:
                    selected_data = selected_data + [BR_receipt_checkboxes_cumsum[j].description]
                    
        # Filter selected data
        df = df[selected_data]
  
        # Transform into date type
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
        
        # Apply filter
        df = df[(df['Date'] >=  pd.Timestamp(begin_period.value)) & (df['Date'] <= pd.Timestamp(end_period.value))] 
        
        # Sort by date
        df = df.sort_values(by = 'Date')
        
        # Refine date
        df['Date'] = df['Date'].dt.strftime('%d/%m/%Y')
        
        # Reset index
        df = df.reset_index(drop =True)
        
        # Red column vector
        red_col = ['Receipt Value', 'FX Value', 'Receipt Value Cum Sum', 'FX Value Cum Sum', 'Settlement Cum Sum']
        
        # Create auxiliar variable
        aux = []
        
        # Loop to define red columns
        for i in range(0,len(red_col)):
            if(red_col[i] in df.columns):
                aux.append(red_col[i])
        if aux == []:
            
            # Print result (without red columns)
            display(df.style.set_precision(2))
            
        else:
            
            # Print result (with red columns)
            display(df.style.applymap(color_negative_red, subset=pd.IndexSlice[:, aux]).set_precision(2))
        
# Create button clicked function
def BR_Receipt_Clean_Clicked(b):
    with BR_receipt_output_clean:
        
        # Clear output_receipt
        BR_receipt_output.clear_output()

In [25]:
##### BRAZIL RECEIPT ON BUTTON CLIKED #####

# On button click
BR_receipt_button.on_click(BR_Receipt_Clicked)
BR_receipt_clean.on_click(BR_Receipt_Clean_Clicked)

In [26]:
##### US TRADES DROPDOWN MENU #####

# Create dropdown menu to selct trade
US_trade_dropdown = interactive(Trade_Select,
                                trade = widgets.Combobox(
                                    options= ['6L F21',
                                             '6L G21',
                                             '6L H21',
                                             '6L J21',
                                             '6L K21',
                                             '6L Q20',
                                             '6L U20',
                                             '6L V20',
                                             '6L X20',
                                             '6L Z20',
                                             'ES H21',
                                             'ES M21',
                                             'ES U20',
                                             'ES Z20',
                                             'MES H21',
                                             'MES M21',
                                             'MES U20',
                                             'MES Z20',
                                             'MS',
                                             'PBR'],
                                    description="Trade:"))

In [27]:
##### US TRADES OPERATIONAL CHECKBOX #####

# Create vector with operations
data = ["Buy Quant", "Buy Price", "Sell Quant", "Sell Price", "Px Close"]

# Create checkbox operations
US_checkboxes_operation = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox operations horizontal box
US_hbox_operation = widgets.HBox(children=US_checkboxes_operation, layout=Layout(width='600px'))

In [28]:
##### US TRADES FINANCIAL CHECKBOX #####

# Create vector with financial
data = ["Hold Pos", "MtoM P&L", "Gross P&L", "Comm", "Exc Fee", "Net P&L"]

# Create checkbox financial
US_checkboxes_financial = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox financial horizontal box
US_hbox_financial = widgets.HBox(children=US_checkboxes_financial, layout=Layout(width='700px'))

In [29]:
##### US TRADES BUTTONS #####

# Create button and output
US_trades_button = Button_Sumbmit()

# Create button output
US_trades_output = widgets.Output()

# Create button and output
US_trades_clean = Button_Clean()

# Create button output
US_trades_output_clean = widgets.Output()

# Unite buttons
US_trades_ui = widgets.HBox([US_trades_button, US_trades_clean])

In [30]:
##### US TRADES BUTTONS FUNCTIONS #####

# Create button clicked function
def US_Trades_Clicked(b):
    with US_trades_output:
        
        # Clean output
        clear_output()

        if (US_trade_dropdown.result is None):
            
            # Display warning message box
            #ctypes.windll.user32.MessageBoxW(0, "Please select a trade!", "Error", 0x10)
            #pymsgbox.alert('This is an alert!', 'Title')
            print('\033[93m' + '\033[1m' + "Please Select a Trade")
        
        else:
            # Create selected_data list
            selected_data = []

            # Fixede variables
            fixed = ['Date', 'Ticker']

            # Add fixed variables
            selected_data = selected_data + fixed

            # Loop operation filter
            for i in range(0, len(US_checkboxes_operation)):
                    if US_checkboxes_operation[i].value == True:
                        selected_data = selected_data + [US_checkboxes_operation[i].description]

            # Loop financial filter
            for j in range(0, len(US_checkboxes_financial)):
                    if US_checkboxes_financial[j].value == True:
                        selected_data = selected_data + [US_checkboxes_financial[j].description]

            # Create trade dataframe
            df = US_trade_dropdown.result

            # Round values
            df = df.round({'Px Close':2, 'Sell Price':2, 'Buy Price': 2, 'Exc Fee':2, 'MtoM P&L':2, 'Gross P&L': 2, 'Net P&L': 2, 'Comm': 2})

            # Change columns type
            df = df.astype({'Sell Quant': int, 'Buy Quant': int, 'Hold Pos': int})

            # Filter data
            df = df[selected_data] 

            # Transform into date type
            df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

            # Apply filter
            df = df[(df['Date'] >=  pd.Timestamp(begin_period.value)) & (df['Date'] <= pd.Timestamp(end_period.value))] 

            # Sort by date
            df = df.sort_values(by = 'Date')

            # Refine date
            df['Date'] = df['Date'].dt.strftime('%d/%m/%Y')

            # Reset index
            df = df.reset_index(drop =True)

            #Create red columns vector
            red_col = ['MtoM P&L' , 'Gross P&L', 'Net P&L', 'Hold Pos']

            # Create auxiliar variable
            aux = []

            # Loop to define red columns
            for i in range(0,len(red_col)):
                if(red_col[i] in df.columns):
                    aux.append(red_col[i])
            if aux == []:

                # Print result (without red columns)
                display(df.style.set_precision(2))

            else:

                # Print result (with red columns)
                display(df.style.applymap(color_negative_red, subset=pd.IndexSlice[:, aux]).set_precision(2))
        
# Create button clicked function
def US_Trades_Clean_Clicked(b):
    with US_trades_output_clean:
        
        # Clear output_receipt
        US_trades_output.clear_output()

In [31]:
##### US TRADES ON BUTTON CLIKED #####

# On button click
US_trades_button.on_click(US_Trades_Clicked)
US_trades_clean.on_click(US_Trades_Clean_Clicked)

In [32]:
###### US TRADES VISUALIZATION BUTTONS #####

# Create button and output
US_trades_viz_button = Button_Sumbmit()

# Create button output
US_trades_viz_output = widgets.Output()

# Create button and output
US_trades_viz_clean = Button_Clean()

# Create button output
US_trades_viz_output_clean = widgets.Output()

# Unite buttons
US_trades_viz_ui = widgets.HBox([US_trades_viz_button, US_trades_viz_clean])

In [33]:
##### US TRADES VISUALIZATION BUTTONS FUNCTIONS #####

# Create button clicked function
def US_Trades_Viz_Clicked(b):  
    with US_trades_viz_output:
        
        # Clean output
        clear_output()
 
        # Create trade dataframe
        df = US_trade_dropdown.result
        
        # Transform into date type
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
        
        # Apply filter
        df = df[(df['Date'] >= pd.Timestamp(begin_period.value)) &  (df['Date'] <= pd.Timestamp(end_period.value))]   
        
        # Transform df format
        df_plot = pd.melt(df[['Buy Quant', 'Sell Quant']])

        # Concat Date
        df_plot['Date'] = pd.concat([df['Date'], df['Date']]).reset_index(drop = True)
        
        # Concat Px Close Diff
        df_plot['Px Close Diff'] = pd.concat([abs(df['Px Close'].diff()), abs(df['Px Close'].diff())]).reset_index(drop = True)
        
        # Plot graph
        fig = px.bar(df_plot, x= "Date", y= "value", color= "variable", title= 'The plot shows trade ') #+ df['Ticker'][0])

        # Update fig layout
        fig.update_layout(barmode= 'group')
        fig.update_yaxes(title='')
        
        # Add close price difference
        fig.add_trace(go.Scatter(mode= "lines", x= df_plot["Date"], y= df_plot["Px Close Diff"], name= "Close Variance"))
        
        # Plot graph
        fig.show()
        
# Create clean button function
def US_Trades_Viz_Clean_Clicked(b):  
    with US_trades_viz_output_clean:
        
        # Clean output
        US_trades_viz_output.clear_output()

In [34]:
##### US TRADES VISUALIZATION ON BUTTON CLIKED #####

# On button click
US_trades_viz_button.on_click(US_Trades_Viz_Clicked)
US_trades_viz_clean.on_click(US_Trades_Viz_Clean_Clicked)

In [35]:
##### US RECEIPT CHECKBOX #####

# Create vector with checkbox data
data = ['Receipt Value', 'FX Description', 'FX Value', 'FX Rate', 'Comm', 'Exc Fee']

# Create checkbox receipt
US_receipt_checkboxes = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox financial horizontal box
US_receipt_hbox = widgets.HBox(children=US_receipt_checkboxes, layout=Layout(width='900px'))

In [36]:
##### US RECEIPT CUMSUM CHECKBOX #####

# Create vector with cumsum
data = ['Receipt Value Cum Sum', 'FX Value Cum Sum', 'Total Cum Sum', 'Settlement Cum Sum']

# Create checkbox cumsum
US_receipt_checkboxes_cumsum = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox cumsum horizontal box
US_receipt_hbox_cumsum = widgets.HBox(children=US_receipt_checkboxes_cumsum, layout=Layout(width='700px'))

In [37]:
##### US RECEIPT BUTTONS #####

# Create button and output
US_receipt_button = Button_Sumbmit()

# Create button output
US_receipt_output = widgets.Output()

# Create button and output
US_receipt_clean = Button_Clean()

# Create button output
US_receipt_output_clean = widgets.Output()

# Unite buttons
US_receipt_ui = widgets.HBox([US_receipt_button, US_receipt_clean])

In [38]:
##### US RECEIPT BUTTONS FUNCTIONS #####

# Create button clicked function
def US_Receipt_Clicked(b):
    with US_receipt_output:
        
        # Clean output
        clear_output()
        
        # Create receipt dataframe
        df = US_futures_receipt
        
        # Create selected_data list
        selected_data = []
        
        # Add fixed variables
        selected_data = selected_data + ['Date']
                    
        # Loop operation filter
        for i in range(0, len(US_receipt_checkboxes)):
                if US_receipt_checkboxes[i].value == True:
                    selected_data = selected_data + [US_receipt_checkboxes[i].description]
                                       
        # Loop financial filter
        for j in range(0, len(US_receipt_checkboxes_cumsum)):
                if US_receipt_checkboxes_cumsum[j].value == True:
                    selected_data = selected_data + [US_receipt_checkboxes_cumsum[j].description]
        
        # Filter selected data
        df = df[selected_data]
  
        # Transform into date type
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
        
        # Apply filter
        df = df[(df['Date'] >=  pd.Timestamp(begin_period.value)) & (df['Date'] <= pd.Timestamp(end_period.value))] 
        
        # Sort by date
        df = df.sort_values(by = 'Date')
        
        # Refine date
        df['Date'] = df['Date'].dt.strftime('%d/%m/%Y')
        
        # Reset index
        df = df.reset_index(drop =True)
        
        # Red column vector
        red_col = ['Receipt Value', 'FX Value', 'Receipt Value Cum Sum', 'FX Value Cum Sum', 'Settlement Cum Sum']
        
        # Create auxiliar variable
        aux = []
        
        # Loop to define red columns
        for i in range(0,len(red_col)):
            if(red_col[i] in df.columns):
                aux.append(red_col[i])
        if aux == []:
            
            # Print result (without red columns)
            display(df.style.set_precision(2))
            
        else:
            
            # Print result (with red columns)
            display(df.style.applymap(color_negative_red, subset=pd.IndexSlice[:, aux]).set_precision(2))
        
# Create button clicked function
def US_Receipt_Clean_Clicked(b):
    with US_receipt_output_clean:
        
        # Clear output_receipt
        US_receipt_output.clear_output()

In [39]:
##### US RECEIPT ON BUTTON CLIKED #####

# On button click
US_receipt_button.on_click(US_Receipt_Clicked)
US_receipt_clean.on_click(US_Receipt_Clean_Clicked)

In [40]:
##### ALL TRADES CHECKBOX #####

# Create vector with checkbox data
data = ['BR Futures', 'US Futures', 'Total Futures', 'US Equities', 'Total', 'Return in %']

# Create checkbox receipt
all_trades_checkboxes = [widgets.Checkbox(value= True, indent= False, description= label) for label in data]

# Create checkbox financial horizontal box
all_trades_hbox = widgets.HBox(children= all_trades_checkboxes, layout= Layout(width= '700px'))

In [41]:
##### ALL TRADES BUTTONS #####

# Create button and output
all_trades_button = Button_Sumbmit()

# Create button output
all_trades_output = widgets.Output()

# Create button and output
all_trades_clean = Button_Clean()

# Create button output
all_trades_output_clean = widgets.Output()

# Unite buttons
all_trades_ui = widgets.HBox([all_trades_button, all_trades_clean])

In [42]:
##### ALL TRADES BUTTONS FUNCTIONS #####

# Create button clicked function
def All_Trades_Clicked(b):
    with all_trades_output:
        
        # Clean output
        clear_output()
        
        # Create receipt dataframe
        df = all_trades
        
        # Create selected_data list
        selected_data = []
        
        # Add fixed variables
        selected_data = selected_data + ['Date']
                    
        # Loop financial filter
        for i in range(0, len(all_trades_checkboxes)):
                if all_trades_checkboxes[i].value == True:
                    selected_data = selected_data + [all_trades_checkboxes[i].description]
        
        # Filter selected data
        df = df[selected_data]
  
        # Transform into date type
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
        
        # Apply filter
        df = df[(df['Date'] >=  pd.Timestamp(begin_period.value)) & (df['Date'] <= pd.Timestamp(end_period.value))] 
        
        # Sort by date
        df = df.sort_values(by = 'Date')
        
        # Refine date
        df['Date'] = df['Date'].dt.strftime('%d/%m/%Y')
        
        # Reset index
        df = df.reset_index(drop =True)
        
        # Red column vector
        red_col = ['BR Futures', 'US Futures', 'Total Futures', 'US Equities', 'Total', 'Return in %']
        
        # Create auxiliar variable
        aux = []
        
        # Loop to define red columns
        for i in range(0,len(red_col)):
            if(red_col[i] in df.columns):
                aux.append(red_col[i])
        if aux == []:
            
            # Print result (without red columns)
            display(df.style.set_precision(2))
            
        else:
            
            # Print result (with red columns)
            display(df.style.applymap(color_negative_red, subset=pd.IndexSlice[:, aux]).set_precision(2))
        
# Create button clicked function
def All_Trades_Clean_Clicked(b):
    with all_trades_output_clean:
        
        # Clear output_receipt
        all_trades_output.clear_output()

In [43]:
##### ALL TRADES ON BUTTON CLIKED #####

# On button click
all_trades_button.on_click(All_Trades_Clicked)
all_trades_clean.on_click(All_Trades_Clean_Clicked)

This report is **interactive**. Therefore, you can make multiple analysis using filters to selects specific data to examine. The report can display financial variables results of the trades and their receipts.

The report uses the following references:

* P&L = Profit and Loss
* MtoM = Market to Market
* Comm = Commisions
* Exc Fee = Exchange Fee
* Reg Fee = Register Fee
* Px Close = Close Price
* Cum Sum = Cumulative Sum

To iniciate the analysis, select the time range you want to exam. After that, use the tabs to choose which trades you want to review.

### Select Time Range:



In [44]:
# Display date range filter
display(begin_period, end_period)

DatePicker(value=datetime.date(2020, 7, 1), description='Start:')

DatePicker(value=datetime.date(2021, 5, 1), description='End:')

In [45]:
##### BRAZIL TRADES ANALYSIS #####

# Create analysis
BR_trade_analysis = widgets.VBox([
    
    # Table
    widgets.HTML('<h1> Brazil Trades Data Analysis </h1>'),
    widgets.HTML('<h2> Trades Data Table </h2>'),
    widgets.HTML('The data shows financial operations and indicators results in <b> BR Real (R$) </b>, calculated for the Brazilian trades using the financial operations displayed.'),
    widgets.HTML('The table can show results from the following trades:'),
    widgets.HTML('&emsp; <i> DOL </i> - Future Comercial Dolar '),
    widgets.HTML('&emsp; <i> WDO </i> - Future Mini Comercial Dolar'),
    widgets.HTML('&emsp; <i> IND </i> - Future Ibovespa'),
    widgets.HTML('&emsp; <i> WIN </i> - Future Mini Ibovespa'),
    widgets.HTML('&emsp; <i> ISP </i> - Future S&P 500'),
    widgets.HTML('&emsp; <i> WSP </i> - Future Micro S&P 500 '),
    widgets.HTML('<br>'), 
    widgets.HTML('<h3> Select Trade: </h3>'), 
    BR_trade_dropdown,
    widgets.HTML('<br>'), 
    widgets.HTML('<h3> Select Operation: </h3>'), 
    BR_hbox_operation,
    widgets.HTML('<h3> Select Indicator: </h3>'), 
    BR_hbox_financial,
    widgets.HTML('<br>'), 
    BR_trades_ui,
    widgets.HTML('<br>'), 
    BR_trades_output,
    BR_trades_output_clean,
    widgets.HTML('<hr>'), 
    
    # Visualization
    widgets.HTML('<h2> Trades Data Visualization </h2>'),
    BR_trades_viz_ui,
    widgets.HTML('<br>'), 
    BR_trades_viz_output,
    BR_trades_viz_output_clean
])

In [46]:
##### BRAZIL RECEIPT ANALYSIS #####

# Create analysis
BR_receipt_analysis = widgets.VBox([
    
    # Table
    widgets.HTML('<h2> Brazil Receipt Table Analysis </h2>'),
    widgets.HTML('The data shows the Brazilian trades recipt values in BR reais (R$). The receipt contains the sum of all trades results'),
    widgets.HTML('Use the checkboxes to display the financial operation and indicator.'),
    widgets.HTML('<br>'), 
    widgets.HTML('<h3> Select Operation: </h3>'), 
    BR_receipt_hbox,
    widgets.HTML('<h3> Select Cumulative Sum: </h3>'), 
    BR_receipt_hbox_cumsum,
    widgets.HTML('<br>'), 
    BR_receipt_ui,
    widgets.HTML('<br>'), 
    BR_receipt_output,
    BR_receipt_output_clean,   
    widgets.HTML('<hr>'), 
    widgets.HTML('<h2> Brazil Receipt Visualization Analysis </h2>')
])

In [47]:
##### USA TRADES ANALYSIS #####

# Create analysis
USA_trade_analysis = widgets.VBox([
    
        # Table
        widgets.HTML('<h2> US Trades Data Analysis </h2>'),
        widgets.HTML('US trades table.'),
        widgets.HTML('<br>'), 
        US_trade_dropdown,
        widgets.HTML('<br>'), 
        widgets.HTML('<h3> Select Operation: </h3>'),
        US_hbox_operation,
        widgets.HTML('<h3> Select Indicator: </h3>'), 
        US_hbox_financial,
        widgets.HTML('<br>'), 
        US_trades_ui,
        widgets.HTML('<br>'), 
        US_trades_output,
        US_trades_output_clean,
        widgets.HTML('<hr>'), 
    
        # Visualization
        widgets.HTML('<h2> US Trades Visualization Analysis </h2>'),
        US_trades_viz_ui,
        widgets.HTML('<br>'), 
        US_trades_viz_output,
        US_trades_viz_output_clean
])

In [48]:
##### USA RECEIPT ANALYSIS #####

# Create analysis
US_receipt_analysis = widgets.VBox([
    
    # Table
    widgets.HTML('<h2> US Receipt Data Analysis </h2>'),
    widgets.HTML('US receipt table.'),
    widgets.HTML('<br>'), 
    widgets.HTML('<h2> Futures Trades </h2>'),
    widgets.HTML('<h3> Select Operation: </h3>'), 
    US_receipt_hbox,
    widgets.HTML('<h3> Select Cumulative Sum: </h3>'), 
    US_receipt_hbox_cumsum,
    widgets.HTML('<br>'), 
    US_receipt_ui,
    widgets.HTML('<br>'), 
    US_receipt_output,
    US_receipt_output_clean,
    widgets.HTML('<hr>'), 
    widgets.HTML('<h2> US Receipt Visualization Analysis </h2>')
])

In [49]:
##### ALL TRADES ANALYSIS #####

# Create analysis
All_trades_analysis = widgets.VBox([
    widgets.HTML('<h2> All Trades Data Analysis </h2>'),
    widgets.HTML('US and BR trades results table.'),
    widgets.HTML('<br>'),
    widgets.HTML('<h3> Select Operation: </h3>'), 
    all_trades_hbox,
    widgets.HTML('<br>'), 
    all_trades_ui,
    widgets.HTML('<br>'), 
    all_trades_output,
    all_trades_output_clean 
])

In [50]:
##### DISPLAY TABS #####

# Create nested tab
tab_nest = widgets.Tab()

# Set each tab analysis
tab_nest.children = [BR_trade_analysis, BR_receipt_analysis, USA_trade_analysis, US_receipt_analysis, All_trades_analysis]

# Set tab title
tab_nest.set_title(0, 'Brazil Trades')
tab_nest.set_title(1, 'Brazil Recipt')
tab_nest.set_title(2, 'US Trades')
tab_nest.set_title(3, 'US Receipt')
tab_nest.set_title(4, 'All')

# Plot tab
tab_nest